In [ ]:
# pip install gym==0.12.5

In [1]:
import numpy as np
import gym

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

env = gym.make('CartPole-v0')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n

model = Sequential()
model.add(Dense(24, input_dim=state_size, activation='relu'))
model.add(Dense(24, activation='relu'))
model.add(Dense(action_size, activation='linear'))
model.compile(loss='mse', optimizer=Adam(lr=0.001))

episodes = 2 # 1000
batch_size = 10
gamma = 0.95
epsilon = 1.0
epsilon_min = 0.01
epsilon_decay = 0.995

for episode in range(episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    done = False
    score = 0
    
    while not done:
        if np.random.rand() <= epsilon:
            action = np.random.randint(0, action_size)
        else:
            q_values = model.predict(state)
            action = np.argmax(q_values[0])
            
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, state_size])
        
        score += reward
        
        target = reward + gamma * np.amax(model.predict(next_state)[0])
        q_values = model.predict(state)
        q_values[0][action] = target
        
        model.fit(state, q_values, verbose=0)
        
        state = next_state
        
        if done:
            if score < 195:
                print(f'Episode: {episode}, Score: {score}, Epsilon: {epsilon}')
            else:
                print(f'Episode: {episode}, Score: {score}')
            if epsilon > epsilon_min:
                epsilon *= epsilon_decay
            break

2023-04-25 09:30:52.423483: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 09:30:52.627406: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-25 09:30:52.627476: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-25 09:30:52.673267: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-04-25 09:30:53.574949: W tensorflow/stream_executor/pla

1/1 [==============================] - 0s 20ms/step
Episode: 0, Score: 42.0, Epsilon: 1.0
1/1 [==============================] - 0s 28ms/step
Episode: 1, Score: 14.0, Epsilon: 0.995
